In [ ]:
!pip install pypulseq==1.3.1.post1 &> /dev/null
!pip install MRzeroCore &> /dev/null
!wget https://github.com/MRsources/MRzero-Core/raw/main/documentation/playground_mr0/numerical_brain_cropped.mat &> /dev/null

(burst_TSE_seq)=
# BURST TSE

https://link.springer.com/article/10.1007/BF02660372

https://pubmed.ncbi.nlm.nih.gov/7984078/

This Burst-TSE follows the main idea of encoding multiple z-states that are then
read in a single readout (with multiple refocusings). It does currently not
minimize gradient switching (phase encoding is applied as short gradient after
every k-space line, instead of continuously). Excitation pulses are constant
flip angles, which is results in varying signal intensity.

In [ ]:
#@title 1. Imports
import numpy as np
# newer numpy versions don't contain this, but pypulseq still relies on it
np.int = int
np.float = float
np.complex = complex

import torch
import matplotlib.pyplot as plt
import MRzeroCore as mr0

res = (64, 64)

data = mr0.VoxelGridPhantom.load_mat('numerical_brain_cropped.mat').interpolate(res[0], res[1], 1).build()

In [ ]:
#@title 2. Burst sequence definition
def build_seq(burst_flip=5, refoc_flip=120, spoiler=10, refocs=8, CPMG=True):
    burst_len = res[1] // refocs
    assert res[1] / refocs % 1 == 0

    t_pulse = 0.5e-3
    t_refoc_pulse = 1e-3
    t_adc = 30e-6

    seq = mr0.Sequence()

    # BURST pulse
    for i in range(burst_len):
        rep = seq.new_rep(2)

        rep.pulse.angle = burst_flip * torch.pi/180
        rep.event_time[0] = t_pulse

        rep.gradm[1, 0] = res[0] + spoiler
        rep.event_time[1] = (res[0] + spoiler) * t_adc


    # TSE readout
    for i in range(refocs):
        rep = seq.new_rep(2 + (res[0] + 1) * burst_len + 1)
        rep.pulse.angle = refoc_flip * torch.pi / 180
        if CPMG == True:
            rep.pulse.phase = (0.5 - (i % 2)) * torch.pi
        elif CPMG == "wrong":
            rep.pulse.phase = torch.pi / 2
        rep.event_time[0] = t_refoc_pulse

        rep.gradm[1, 0] = spoiler + res[0] // 2 - 1
        rep.gradm[1, 1] = -res[0] // 2 + i * burst_len
        rep.event_time[1] = rep.gradm[1, 0] * t_adc

        # Readout
        for j in range(burst_len):
            start = 2 + j * (res[0] + 1)
            stop = start + res[0]

            rep.gradm[start:stop, 0] = 1
            rep.adc_usage[start:stop] = 1
            rep.event_time[start:stop] = t_adc

            rep.gradm[stop, 0] = spoiler
            rep.gradm[stop, 1] = 1
            rep.event_time[stop] = spoiler * t_adc

        rep.gradm[-1, :] = -rep.gradm[:-1, :].sum(0)
        rep.event_time[-1] = rep.gradm[-1, 0] * t_adc

    return seq

In [ ]:
#@title Simulate
# Use less states / isochromats here because free google Colab is slow.
# This means that isochromats will not produce a usable image!

spin_recos = {}
pdg_recos = {}

for i in range(3):
    cpmg = [False, "wrong", True][i]
    seq = build_seq(CPMG=cpmg)

    # Very high thresholds for inaccurate but fast doc build
    graph = mr0.compute_graph(seq, data, 1000, 0.01)
    signal = mr0.execute_graph(graph, seq, data, 0.1, 0.1, print_progress=False)
    pdg_recos[cpmg] = torch.fft.fftshift(torch.fft.fft2(signal.view(res)))

    # NOTE: Commented for documentation build out because slow
    # signal = mr0.isochromat_sim(seq, data, 100).cpu().flatten()
    # spin_recos[cpmg] = torch.fft.fftshift(torch.fft.fft2(signal.view(res)))
    spin_recos[cpmg] = torch.zeros_like(pdg_recos[cpmg])

In [ ]:
#@title Plot the figure
text_args = {"c": "w", "fontsize": 14, "ha": "center", "va": "center", "bbox": {"fill": True}}

plt.figure(figsize=(9, 6), dpi=100)
for i in range(3):
    cpmg = [False, "wrong", True][i]
    name = ["CPMG violated", "CPMG", "CPMG ($\\pm 180°$)"][i]

    plt.subplot(231 + i)
    plt.text(32, 58, name, text_args)
    if i == 0:
        plt.text(5, 32, "PDG", text_args, rotation="vertical")
    plt.imshow(pdg_recos[cpmg].abs(), origin="lower", vmin=0, vmax=300)
    plt.axis("off")
    plt.subplot(234 + i)
    if i == 0:
        plt.text(5, 32, "Isochromats", text_args, rotation="vertical")
    plt.imshow(spin_recos[cpmg].abs(), origin="lower", vmin=0, vmax=300)
    plt.axis("off")
plt.subplots_adjust(hspace=0.05, wspace=0)
plt.show()

In [ ]:
#@title Tau-view of the PDG

logarithmic = True  # @param {type: "boolean"}
what = "emitted signal"  #@param ["emitted signal", "latent signal", "magnetization"]
sim_only = False  # @param {type: "boolean"}
eps = 1e-7

values = []  # (rep, dephasing, weight or signal)

for i, rep in enumerate(graph):
    for state in rep:
        if sim_only and state.kt_vec is None:
            continue
        if what == "emitted signal" and state.dist_type == '+':
            values.append((
                i,
                state.prepass_kt_vec[3],
                state.emitted_signal + eps,
            ))
        elif what == "latent signal":
            values.append((
                i,
                state.prepass_kt_vec[3],
                state.latent_signal + eps,
            ))
        elif what == "magnetization":
            values.append((
                i,
                state.prepass_kt_vec[3],
                np.abs(state.prepass_mag) + eps,
            ))

values = sorted(values, key=lambda v: v[2])
r = [v[0] for v in values]
t = [v[1] for v in values]
if logarithmic:
    v = [np.log10(v[2]) for v in values]
else:
    v = [v[2] for v in values]

plt.figure()
if logarithmic:
    plt.scatter(r, t, c=v, s=20, zorder=10, vmin=np.log10(eps), vmax=0)
else:
    plt.scatter(r, t, c=v, s=20, zorder=10, vmin=0, vmax=1)

plt.grid()
plt.colorbar()

plt.xlabel("Repetition")
plt.ylabel("$\\tau$ dephasing [s]")
plt.show()